<a href="https://colab.research.google.com/github/ShambhaviCodes/Text-Classification-using-ELMo-Embedding/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Dataset is the BBC News Dataset in which we have to classify the texts into 5 classes of the genre of news they belong to.

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1_1yMSh-oswCwQbim_trNmCo6HqJ4jLWo'

In [ ]:
fluff, id = link.split('=')
print (id)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import os

In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
data = pd.read_csv('Filename.csv')
# Dataset is now stored in a Pandas Dataframe

In [ ]:
from sklearn.preprocessing import LabelEncoder
df3 = pd.DataFrame()
df3["text"] = data["text"]
df3["label"] = LabelEncoder().fit_transform(data["category"])

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
df3['text'] = df3['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df3['text'].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0    tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv radically different five years time. according expert panel g...
1    worldcom boss left books alone former worldcom boss bernie ebbers accused overseeing $11bn (£5.8bn) fraud never made accounting decisions witness told jurors. david myers made comments questioning...
2    tigers wary farrell gamble leicester say rushed making bid andy farrell great britain rugby league captain decide switch codes. anybody else involved process still way away going next stage tigers...
3    yeading face newcastle fa cup premiership side newcastle united face trip ryman premier league leaders yeading fa cup third round. game - arguably highlight draw - potential money-spinner non-leag...
4    ocean twelve raids box office ocean twelve crime caper sequel starring george clooney brad pitt julia roberts gone straight number one us box office chart. took $40.8m (£21m) 

In [ ]:
df3['text'].head()

0    tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv radically different five years time. according expert panel g...
1    worldcom boss left books alone former worldcom boss bernie ebbers accused overseeing $11bn (£5.8bn) fraud never made accounting decisions witness told jurors. david myers made comments questioning...
2    tigers wary farrell gamble leicester say rushed making bid andy farrell great britain rugby league captain decide switch codes. anybody else involved process still way away going next stage tigers...
3    yeading face newcastle fa cup premiership side newcastle united face trip ryman premier league leaders yeading fa cup third round. game - arguably highlight draw - potential money-spinner non-leag...
4    ocean twelve raids box office ocean twelve crime caper sequel starring george clooney brad pitt julia roberts gone straight number one us box office chart. took $40.8m (£21m) 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
from sklearn import preprocessing
import keras
y = list(df3['label'])
x = list(df3['text'])

en = preprocessing.LabelEncoder()
en.fit(y)

def encode(en, labels):
    enc = en.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(en, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return en.inverse_transform(dec)


x_enc = x
y_enc = encode(en, y)

Using TensorFlow backend.


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.3, random_state=88)

In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(5, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=16)
    model.save_weights('./elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=16)

y_test = decode(en, y_test)
y_preds = decode(en, predicts)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1



1557/1557 [==============================] - 8377s 5s/step - loss: 0.2890 - acc: 0.9146
